In [1]:
from Bio import SeqIO, motifs
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import csv
from Bio.SeqRecord import SeqRecord
import torch
import json
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import StandardScaler



In [3]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
import numpy as np
from sklearn.decomposition import PCA

In [4]:
def scale_data(train_data, test_data):
    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data)
    scaled_test_data = scaler.transform(test_data)
    return pd.DataFrame(scaled_train_data, columns=train_data.columns), pd.DataFrame(scaled_test_data, columns=test_data.columns)

In [ ]:
training_data = pd.read_csv("df_train_feature_pb.csv", index_col=0)
testing_data = pd.read_csv("df_test_feature_pb.csv", index_col=0)

In [4]:
# Separate features and labels
X_train_data = training_data.drop('Label', axis=1)
y_train_data = training_data['Label']
X_test_data = testing_data.drop('Label', axis=1)
y_test_data = testing_data['Label']

In [5]:
#scaled_X_train, scaled_X_test = scale_data(X_train_data, X_test_data)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
import numpy as np

2024-01-26 13:45:56.105877: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-26 13:45:56.107818: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-26 13:45:56.132608: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 13:45:56.132626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 13:45:56.133437: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [7]:
from sklearn.decomposition import PCA

# Define the number of features you want after dimensionality reduction
n_components = 32 * 32

pca = PCA(n_components=n_components)

X_train_ini = training_data.drop('Label', axis=1).values
X_train_pca = pca.fit_transform(X_train_ini)  # Apply PCA to training data

X_test_ini = testing_data.drop('Label', axis=1).values
X_test_pca = pca.transform(X_test_ini)  # Apply PCA to testing data

X_train = X_train_pca.reshape(-1, 32, 32, 1)
y_train = to_categorical(training_data['Label'].values, num_classes=2)

X_test = X_test_pca.reshape(-1, 32, 32, 1)
y_test = to_categorical(testing_data['Label'].values, num_classes=2)

In [16]:
len(X_train)

2518

In [17]:
len(X_train[0])

32

In [20]:
# 32*32
model = Sequential()
model.add(Conv2D(32, kernel_size=3, input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(64, kernel_size=3))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(128, kernel_size=3))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid')) 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)


Epoch 1/50
79/79 [==============================] - 1s 10ms/step - loss: 0.6433 - accuracy: 0.6311 - val_loss: 0.5733 - val_accuracy: 0.7365
Epoch 2/50
79/79 [==============================] - 1s 10ms/step - loss: 0.5688 - accuracy: 0.7073 - val_loss: 0.5345 - val_accuracy: 0.7556
Epoch 3/50
79/79 [==============================] - 1s 10ms/step - loss: 0.5391 - accuracy: 0.7315 - val_loss: 0.5244 - val_accuracy: 0.7714
Epoch 4/50
79/79 [==============================] - 1s 10ms/step - loss: 0.5180 - accuracy: 0.7423 - val_loss: 0.5966 - val_accuracy: 0.7413
Epoch 5/50
79/79 [==============================] - 1s 10ms/step - loss: 0.5120 - accuracy: 0.7411 - val_loss: 0.5034 - val_accuracy: 0.7778
Epoch 6/50
79/79 [==============================] - 1s 10ms/step - loss: 0.4892 - accuracy: 0.7665 - val_loss: 0.5106 - val_accuracy: 0.7746
Epoch 7/50
79/79 [==============================] - 1s 10ms/step - loss: 0.4891 - accuracy: 0.7585 - val_loss: 0.5256 - val_accuracy: 0.7635
Epoch 8/50
79

In [21]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred_classes == y_true)
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f1 = f1_score(y_true, y_pred_classes)
mcc = matthews_corrcoef(y_true, y_pred_classes)
auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'Matthews Correlation Coefficient: {mcc}')
print(f'AUC: {auc}')


20/20 [==============================] - 0s 3ms/step
Accuracy: 0.7603174603174603
Precision: 0.731638418079096
Recall: 0.8222222222222222
F1-Score: 0.7742899850523169
Matthews Correlation Coefficient: 0.5246717439281506
AUC: 0.8261224489795919
